In [1]:
import numpy as np
import itertools
import copy

from scipy.optimize import fsolve
from scipy.special import gamma

In [2]:
class City(object):
    
    def __init__(self, width, height, hospital_count, subregion_count):
        
        self.width = width
        self.height = height
        #for simplicity, subregion_count = base_count
        
        hospital_x = np.linspace(0, self.width, num = hospital_count + 2)
        self.hospital_location = [(hospital_x[i+1], self.height/2) for i in range(hospital_count)]
        
        self.subregion_count = subregion_count + (subregion_count % 2)
        subregion_x = np.linspace(0, self.width, num = int(self.subregion_count/2 + 2))
        subregion_y = np.linspace(0, self.height, num = 4)
        base_location = [[(subregion_x[i+1], subregion_y[j+1]) for j in range(2)] for i in range(int(self.subregion_count/2))]
        self.base_location = list(itertools.chain(*base_location))
        

In [3]:
def gen_call_arrival_loc(city):
    return (np.random.uniform(0, city.width),np.random.uniform(0, city.height))

In [4]:
def gen_call_arrival_time():
    return np.random.exponential(10)

In [5]:
def gen_call_priority():
    return int(np.random.uniform(0,1) < 0.25)

In [6]:
def gen_atlocation_service_time():
    return np.random.exponential(12)

In [7]:
def weibull_param_relationship(scale, mean, stddev):
    return stddev**2/mean**2 - gamma(1 + 2/scale)/(gamma(1+1/scale)**2) + 1

def get_weibull_parameters(mean = 30, stddev = 13):
    scale = fsolve(weibull_param_relationship, 1,args=(mean, stddev))
    shape = mean / gamma(1+1/scale)
    
    return scale, shape

def gen_athospital_service_time():
    scale, shape = get_weibull_parameters(mean = 30, stddev = 13)
    return scale * np.random.weibull(shape)

In [8]:
class Call(object):
    
    def __init__(self, location, time_arrival, interarrival_time, priority = 0):
        
        self.location = location
        
        #initialize the call to be unassigned
        #-1-unassigned, 0 to (N-1) assigned ambulance index 
        self.status = -1
        self.arrival_time = time_arrival
        self.next_arrival_time = self.arrival_time + interarrival_time
        
        #0-low priority, 1-high priority
        self.priority = priority

In [9]:
def get_distance(location1, location2):
    
    distance = np.abs(location1[0] - location2[0]) + np.abs(location1[1] - location2[1])
    return distance

In [10]:
def get_ambulance_travel_time(distance, speed = 10):
    return distance/speed

In [11]:
class Ambulance(object):
    
    def __init__(self, base):
        
        #fix the ambulance's home base
        self.base = base
        
        #initialize the ambulance idle at home base
        
        #status code: 0-idle at base, 1-going to scene of call, 2-serving at scene of call, 3-going to hospital
        #4-transferring patient at hospital, 5-returning to base
        self.status = 0
        self.origin = base
        self.destination = base # if destination = origin, ambulance is stationary
        self.time = 0
        self.endtime = np.inf
        
    def update_status(self, status, origin, destination, time, endtime):
        self.status = status
        
        self.origin = origin
        self.destination = destination
        self.time = time
        self.endtime = endtime
        
    def redployment(self):
        
        #current heuristic: return to home base
        distance = get_distance(self.origin, self.base)
        travel_time = get_ambulance_travel_time(distance)
        self.update_status(5, self.origin, self.base, copy.deepcopy(self.endtime), copy.deepcopy(self.endtime) + travel_time)
        
    def return_to_base(self):
        self.update_status(0, self.base, self.base, copy.deepcopy(self.endtime), np.inf)
        
        
        
    def assign_to_call(self, call, time, index):
        
        distance = get_distance(self.origin, call.location)
        travel_time = get_ambulance_travel_time(distance)
        self.update_status(1, self.origin, call.location, time, time + travel_time)
        
        call.status = index
        
        
    def reach_call_location(self, call, callList):
    
        atlocation_servicetime = gen_atlocation_service_time()
        self.update_status(2, call.location, call.location, 
                           copy.deepcopy(self.endtime), copy.deepcopy(self.endtime) + atlocation_servicetime)

        callList.remove(call)
    
    def transport_to_hospital(self, city):
        
        #transport to nearest hospital
        
        hospital_list = city.hospital_location
        
        min_distance = city.width + city.height
        nearest_hospital = hospital_list[0]
        for hospital in hospital_list:
            distance = get_distance(self.origin, hospital)
            if distance < min_distance:
                min_distance = distance
                nearest_hospital = hospital
        
        travel_time = get_ambulance_travel_time(min_distance)
        self.update_status(3, self.origin, hospital, 
                           copy.deepcopy(self.endtime), copy.deepcopy(self.endtime) + travel_time)
        
    def reach_hospital(self):
        hospital_servicetime = gen_athospital_service_time()
        self.update_status(4, copy.deepcopy(self.destination), copy.deepcopy(self.destination), 
                           copy.deepcopy(self.endtime), copy.deepcopy(self.endtime) + hospital_servicetime)




In [12]:
def arrival(ambulanceList, callList, time_arrival, M):
    
    call = Call(gen_call_arrival_loc(city), time_arrival, gen_call_arrival_time(), priority = gen_call_priority())
    
    if len(callList) >= M:
        print("New call arrived. No more capacity. Reject call.")
    
    else:
    
        print("New call arrived. Capacity available. Add to call list.")
        callList.append(call)

        min_distance = 15
        assign = -1
        index = 0

        nearest_distance = 15
        for ambulance in ambulanceList:

            if ambulance.status == 0:
                distance = get_distance(ambulance.origin, call.location)
                if distance < nearest_distance:
                    nearest_distance = distance
                    assign = index

            index = index + 1

        if assign > -1:
            # when the call arrives, there are ambulances idle at base, so assign the call to the nearest ambulance
            print("Idle ambulance at base available. Assign Ambulance", assign, ". Now travelling to call location.")
            ambulanceList[assign].assign_to_call(call, time_arrival, assign)

        else:
            print("No available ambulance at the moment.")
            
    time_arrival = call.next_arrival_time
    
    return ambulanceList, callList, time_arrival

In [13]:
def get_first_unassigned(callList, M):
    
    # Assign based on FCFS (regardless of priority)
    assign = -1
    index = 0
    
    for call in callList:
        if call.status == -1:
            assign = index
        
        if assign > -1:
            break
        
        index = index + 1
        
    if index == M:
        print("No unassigned call.")
        return index
    else:
        print("There is unassigned call (policy disregard priority).")
        return assign

In [14]:
def get_first_highpriority_unassigned(callList, M):
    
    # Assign based on FCFS within priority
    # assume only high and low, two levels or priority
    high_assign = -1
    low_assign = -1
    index = 0
    
    for call in callList:
        if call.status == -1:
            
            if call.priority == 1:
                high_assign = index
                break
                
            elif call.priority == 0:
                low_assign = index
        
        index = index + 1
        
    if index < M:
        print("There is unassigned high priority call.")
        return high_assign
    else:
        if low_assign > -1:
            print("No high priority call waiting, but there is unassigned low priority call.")
            return low_assign
        else:
            print("No unassigned call.")
            return index

In [15]:
def get_assigned_call(am_index, callList):
    
    call_assign = -1 # no call is assigned to this ambulance_index
    
    call_id = 0
    for call in callList:
        if call.status == am_index:
            call_assign = call_id
            break

        call_id = call_id + 1

    return call_assign

In [16]:
def get_next_event(time_arrival, ambulanceList, callList, city, M, policy):
    
    ambulanceEndTime_min = np.inf
    index_min = -1
    index = 0
    for ambulance in ambulanceList:
        if ambulance.endtime < ambulanceEndTime_min:
            ambulanceEndTime_min = copy.deepcopy(ambulance.endtime)
            index_min = index
        
        index = index + 1
    
    next_event_time = min(time_arrival, ambulanceEndTime_min)
    
    if next_event_time == time_arrival:
        # print("New call arrived.")
        ambulanceList, callList, time_arrival = arrival(ambulanceList, callList, time_arrival, M)
        
    else:
        if ambulanceList[index_min].status == 1:
            print("Ambulance", index_min, " reach call location. Start at-location treatment. Remove call from call list.")
            
            call_assign = get_assigned_call(index_min, callList)
            
            call = callList[call_assign]
            ambulanceList[index_min].reach_call_location(call, callList)
            
        elif ambulanceList[index_min].status == 2:
            print("Ambulance", index_min, " finish at-location treatment. Start going to hospital.")
            ambulanceList[index_min].transport_to_hospital(city)
            
        elif ambulanceList[index_min].status == 3:
            print("Ambulance", index_min, " reach hospital. Start transferring patient to hospital.")
            ambulanceList[index_min].reach_hospital()
            
        elif ambulanceList[index_min].status == 4:
            
            print("Ambulance", index_min, " finish transfering patient to hospital. Decide next step (assign to call or return to base).")
            if len(callList) == 0:
                print("Return to base.")
                ambulanceList[index_min].redployment()
            else:
                print("There is a call list.")
                #policy is a function passed in 
                index = policy(callList, M)
                
                if index == M:
                    # calls in callList have all been assigned with an ambulance
                    print("Return to base.")
                    ambulanceList[index_min].redployment()
                else:
                    # there is a call in list unassigned, waiting for service, thus assign ambulance to call
                    print("Assign to call according to policy.")
                    ambulanceList[index_min].assign_to_call(callList[index], next_event_time, index_min)
                    
        elif ambulanceList[index_min].status == 5:
            print("Ambulance", index_min, " reach base. Start idling.")
            ambulanceList[index_min].return_to_base()
    
    return time_arrival, ambulanceList, callList, next_event_time

In [17]:
def print_ambulance(ambulanceList):
    statusList = []
    originList = []
    destinationList = []
    timeList = []
    endtimeList = []
    
    for ambulance in ambulanceList:
        statusList.append(ambulance.status)
        originList.append(ambulance.origin)
        destinationList.append(ambulance.destination)
        timeList.append(ambulance.time)
        endtimeList.append(ambulance.endtime)
    
    return statusList, originList, destinationList, timeList, endtimeList
    

In [18]:
def print_call(callList):
    
    statusList = []
    locationList = []
    timeList = []
    
    for call in callList:
        statusList.append(call.status)
        locationList.append(call.location)
        timeList.append(call.arrival_time)
    
    return statusList, locationList, timeList

# Set up the City

In [19]:
city = City(10, 5, 2, 4)
ambulanceList = [Ambulance(city.base_location[i]) for i in range(len(city.base_location))]
callList = []
time_arrival = 0

# Repetitively run the following two blocks to visualize the process

In [20]:
time = 0
horizon = 100

while time < horizon:
    time_arrival, ambulanceList, callList, time = get_next_event(time_arrival, ambulanceList, callList, city, 100, get_first_highpriority_unassigned)
    # print("Call List Length: ", len(callList))

New call arrived. Capacity available. Add to call list.
Idle ambulance at base available. Assign Ambulance 2 . Now travelling to call location.
New call arrived. Capacity available. Add to call list.
Idle ambulance at base available. Assign Ambulance 0 . Now travelling to call location.
Ambulance 0  reach call location. Start at-location treatment. Remove call from call list.
Ambulance 2  reach call location. Start at-location treatment. Remove call from call list.
Ambulance 2  finish at-location treatment. Start going to hospital.
Ambulance 2  reach hospital. Start transferring patient to hospital.
New call arrived. Capacity available. Add to call list.
Idle ambulance at base available. Assign Ambulance 1 . Now travelling to call location.
Ambulance 1  reach call location. Start at-location treatment. Remove call from call list.
Ambulance 2  finish transfering patient to hospital. Decide next step (assign to call or return to base).
Return to base.
Ambulance 2  reach base. Start idlin

In [21]:
am_statusList, am_originList, am_destinationList, am_timeList, am_endtimeList = print_ambulance(ambulanceList)
cl_statusList, cl_locationList, cl_timeList = print_call(callList)

print("Next Arrival: ", time_arrival)

print("Ambulance List Print: ")
print("Status List: ", am_statusList)
print("Origin List: ", am_originList)
print("Destination List: ", am_destinationList)
print("Event start time List: ", am_timeList)
print("Event end time List: ", am_endtimeList)

print("Call List Print: ")
print("Status List: ", cl_statusList)
print("Location List: ", cl_locationList)
print("Arrival time List: ", cl_timeList)

Next Arrival:  104.6113453851732
Ambulance List Print: 
Status List:  [0, 1, 0, 0]
Origin List:  [(3.3333333333333335, 1.6666666666666667), (3.3333333333333335, 3.3333333333333335), (6.666666666666667, 1.6666666666666667), (6.666666666666667, 3.3333333333333335)]
Destination List:  [(3.3333333333333335, 1.6666666666666667), (1.3457622634140765, 3.7125844847836627), (6.666666666666667, 1.6666666666666667), (6.666666666666667, 3.3333333333333335)]
Event start time List:  [array([79.66683383]), 100.53445299732965, array([94.97240765]), array([55.94067992])]
Event end time List:  [inf, 100.77113521946661, inf, inf]
Call List Print: 
Status List:  [1]
Location List:  [(1.3457622634140765, 3.7125844847836627)]
Arrival time List:  [100.53445299732965]


In [22]:
# time_arrival, ambulanceList, callList, time = get_next_event(time_arrival, ambulanceList, callList, city, 3, get_first_unassigned)
# am_statusList, am_originList, am_destinationList, am_timeList, am_endtimeList = print_ambulance(ambulanceList)
# cl_statusList, cl_locationList, cl_timeList = print_call(callList)

# print("Next Arrival: ", time_arrival)

# print("Ambulance List Print: ")
# print("Status List: ", am_statusList)
# print("Origin List: ", am_originList)
# print("Destination List: ", am_destinationList)
# print("Event start time List: ", am_timeList)
# print("Event end time List: ", am_endtimeList)

# print("Call List Print: ")
# print("Status List: ", cl_statusList)
# print("Location List: ", cl_locationList)
# print("Arrival time List: ", cl_timeList)

In [23]:
# time_arrival, ambulanceList, callList = get_next_event(time_arrival, ambulanceList, callList, city, 3)
# am_statusList, am_originList, am_destinationList, am_timeList, am_endtimeList = print_ambulance(ambulanceList)
# cl_statusList, cl_locationList, cl_timeList = print_call(callList)

# print("Next Arrival: ", time_arrival)

# print("Ambulance List Print: ")
# print("Status List: ", am_statusList)
# print("Origin List: ", am_originList)
# print("Destination List: ", am_destinationList)
# print("Event start time List: ", am_timeList)
# print("Event end time List: ", am_endtimeList)

# print("Call List Print: ")
# print("Status List: ", cl_statusList)
# print("Location List: ", cl_locationList)
# print("Arrival time List: ", cl_timeList)